# IBM DATA SCIENCE CAPSTONE PROJECT
By Nirav Patel



### Librarys

In [11]:
!pip install geopy
!pip install BeautifulSoup4
!pip install lxml
import numpy as np
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import geopy.geocoders
import folium
from bs4 import BeautifulSoup

## Import data from Available files
We already analysed and clustered the data for the Toronto city in lab session earlier.
Here in this project, I am not explaining all the steps but just quickly finish the file.
I will just import the data sheet for the toronto with the latitude, longitude and neighborhood with the postal codes included as shown below.

In [13]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(url,features='html')
table=soup.find('table',{'class':'wikitable sortable'})
ths=table.find_all('th')
headings = [th.text.strip() for th in ths]

In [16]:
Postcode=[]
Borough=[]
Neighbourhood=[]
for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode_, Borough_, Neighbourhood_=[td.text.strip() for td in tds[:3]]
        Postcode.append(Postcode_)
        Borough.append(Borough_)
        Neighbourhood.append(Neighbourhood_)

In [51]:
df=pd.DataFrame(columns=headings)
df['Postcode']=Postcode
df['Neighbourhood']=Neighbourhood
df['Borough']=Borough
df=df[df.Borough!='Not assigned']
df['Neighbourhood']=df['Neighbourhood'].replace('Not assigned', df['Borough'])
grpby=df.groupby(['Postcode','Borough'], as_index=False)
df=grpby.agg(lambda x: ', '.join(x))
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [54]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')

df=pd.concat([df, df_geo], join='inner', axis=1)
df.drop('Postal Code', inplace=True, axis=1)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476


In [24]:
final_data = df[(df['Borough']=='Downtown Toronto') | 
               (df['Borough']=='East Toronto') | 
               (df['Borough']=='West Toronto')|
               (df['Borough']=='Central Toronto')|
              (df['Borough'] == 'Scarborough')]

final_data.reset_index(inplace=True)
final_data.drop('index',1,inplace=True)
final_data.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Plot
We will plot the data on the toronto map to have an idea of locations over the city

In [25]:
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_data['Latitude'], final_data['Longitude'], final_data['Borough'], final_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

Using Foursquare Api and my personal credentials.

In [26]:
CLIENT_ID = '5QIS2WMSBLL2LU0CI1TWRVZWOBZFKRDO4MWLWGIMXMQRVLP0' # your Foursquare ID
CLIENT_SECRET = 'SVKOA1RFJSK2KSUCGF1BDVXPG1KZVZ1VV1PAEZ00MGS0EUQZ' # your Foursquare Secret
VERSION = '20190815' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5QIS2WMSBLL2LU0CI1TWRVZWOBZFKRDO4MWLWGIMXMQRVLP0
CLIENT_SECRET:SVKOA1RFJSK2KSUCGF1BDVXPG1KZVZ1VV1PAEZ00MGS0EUQZ


## Getting nearby Venues
with the radius of 500, we will plot the nearby venues over the map of toronto

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
LIMIT = 100
radius= 500
Toronto_venues = getNearbyVenues(names=final_data['Neighbourhood'],
                                   latitudes=final_data['Latitude'],
                                   longitudes=final_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Co

In [31]:
venues_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
Toronto_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.01
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.00
4,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
5,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
7,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
8,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022222,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,Cedarbrae,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


### Filtering
Now we will filter the venues to just 10. Out of 54 we will select just top ten and procced with the analysis.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Asian Restaurant,Breakfast Spot,Gym,American Restaurant,Hotel
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
3,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Steakhouse,Farmers Market,Cheese Shop,Seafood Restaurant,Bakery,Café,Shopping Mall
4,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


## Use of clusters
Clusters will be created by similarity in top venues

In [34]:
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ = np.append(kmeans.labels_,[2])
kmeans.labels_

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 2, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2])

In [36]:
Toronto_merged = final_data
Toronto_merged['Cluster Labels'] = kmeans.labels_
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
Toronto_merged.head()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,Comic Shop,Fish & Chips Shop,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,History Museum,Bar,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Mexican Restaurant,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Intersection,Rental Car Location,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Indian Restaurant,Korean Restaurant,Women's Store,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fried Chicken Joint,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Electronics Store,Dumpling Restaurant,Eastern European Restaurant


### Plot
Now we will create Map with clusters

In [39]:
# Create Map

toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Finding the best neighbourhood
we find in which Cluster is the Roselawn

In [41]:
Toronto_merged[Toronto_merged['Neighbourhood']=='Roselawn']

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Garden,Women's Store,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Roselawn is in cluster 0 that means the requirements for our client is in the cluster 0 which is nothing but the Borough of Scarborough

In [42]:
Candidates = Toronto_merged.loc[(Toronto_merged['Cluster Labels'] == 0) & (Toronto_merged['Borough'] =='Scarborough')]
Candidates.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,Comic Shop,Fish & Chips Shop,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,History Museum,Bar,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Indian Restaurant,Korean Restaurant,Women's Store,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fried Chicken Joint,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Electronics Store,Dumpling Restaurant,Eastern European Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Mapping the cluster with scarborough area

In [44]:
map_clusters = folium.Map(location=[toronto_latitude,toronto_longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Candidates['Latitude'], Candidates['Longitude'], Candidates['Neighbourhood'], Candidates['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

the last condition to satisfy is to have grocery stores nearby the chosen locatoin.

In [45]:
a=((Candidates=="Grocery Store"))
np.sum(a,axis=1)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


0     0
1     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    1
16    0
dtype: int64

From this result, we know that only candidate (location) that satisfies the conditions is location or candidate number 5

In [46]:
search_query = 'Grocery Store'
radius = 5000
LIMIT = 10
latitude = Candidates.loc[5,'Latitude']
longitude = Candidates.loc[5,'Longitude']
print(latitude,longitude)

43.7447342 -79.23947609999999


In [47]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.sort_values(by='location.distance',inplace=True)
dataframe

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
1,50942b90e4b0a70261f30d18,True Discount Convenience Store,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",v-1565988818,False,1355 Danforth Road,NaN,43.743854,-79.244650,"[{'label': 'display', 'lat': 43.74385420196717...",427,NaN,CA,Scarborough,ON,Canada,"[1355 Danforth Road, Scarborough ON, Canada]",NaN
2,4d5c96f5e7f75481e5221c97,Daisy Mart Convenience Store,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",v-1565988818,False,1351 Danforth Rd,NaN,43.743780,-79.244644,"[{'label': 'display', 'lat': 43.74378, 'lng': ...",428,M1J 1G2,CA,Toronto,ON,Canada,"[1351 Danforth Rd, Toronto ON M1J 1G2, Canada]",NaN
5,4bb3e866f187a59379a013f8,Mac's Convenience Store,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",v-1565988818,False,2900 Eglinton Ave East,at McCowan Road,43.738615,-79.238576,"[{'label': 'display', 'lat': 43.73861453638707...",685,M1J 2E4,CA,Toronto,ON,Canada,"[2900 Eglinton Ave East (at McCowan Road), Tor...",NaN
0,4b2405e2f964a5204b5f24e3,The Beer Store,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",v-1565988818,False,2727 Eglinton Ave E,at Brimley Rd.,43.735941,-79.248805,"[{'label': 'display', 'lat': 43.7359411, 'lng'...",1233,M1K 2S2,CA,Scarborough,ON,Canada,"[2727 Eglinton Ave E (at Brimley Rd.), Scarbor...",NaN
9,4b677b90f964a520a6512be3,The Beer Store,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",v-1565988818,False,3205 Eglinton Ave E,Markham Road,43.743348,-79.218232,"[{'label': 'display', 'lat': 43.7433482, 'lng'...",1715,M1J 2H7,CA,Scarborough,ON,Canada,"[3205 Eglinton Ave E (Markham Road), Scarborou...",NaN
7,4b9bb620f964a520a01c36e3,The Beer Store,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",v-1565988818,False,3561 Lawrence Ave E,Lawrence & Markham,43.759714,-79.225487,"[{'label': 'display', 'lat': 43.7597137, 'lng'...",2011,M1H 1B2,CA,Scarborough,ON,Canada,"[3561 Lawrence Ave E (Lawrence & Markham), Sca...",NaN
6,4b893ecdf964a520a72532e3,NIKE Employee Store,"[{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...",v-1565988818,False,260 Brimley Rd.,NaN,43.726400,-79.246799,"[{'label': 'display', 'lat': 43.72639971565108...",2124,NaN,CA,Scarborough,ON,Canada,"[260 Brimley Rd., Scarborough ON, Canada]",NaN
4,4bfebef9e584c928402b6d25,Salvation Army Thrift Store,"[{'id': '4bf58dd8d48988d101951735', 'name': 'T...",v-1565988818,False,1225 Kennedy Rd,NaN,43.755782,-79.276208,"[{'label': 'display', 'lat': 43.75578210550947...",3199,M1P 2L4,CA,Scarborough,ON,Canada,"[1225 Kennedy Rd, Scarborough ON M1P 2L4, Canada]",NaN
3,5085ec39e4b0b1ead2eb0818,Disney Store,"[{'id': '4bf58dd8d48988d1f3941735', 'name': 'T...",v-1565988818,False,300 Borough Drive,in Scarborough Town Centre,43.775537,-79.256833,"[{'label': 'display', 'lat': 43.775537, 'lng':...",3702,M1P 4P5,CA,Scarborough,ON,Canada,[300 Borough Drive (in Scarborough Town Centre...,NaN
8,4d1127d65f3376eb86661386,Tommy Hilfiger Company Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1565988818,False,300 Borough Dr,NaN,43.776015,-79.257369,"[{'label': 'display', 'lat': 43.77601546103458...",3767,M1P 4P5,CA,Scarborough,ON,Canada,"[300 Borough Dr, Scarborough ON M1P 4P5, Canada]",Scarborough City Centre


Finally, we analyze the ratings of the groceries stores in a radius of 500 for the candidate chosen

In [49]:
def rate(id):
    venue_id = id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')
        
dataframe['id'].apply(lambda id: rate(id) )

This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
6.4
6.1
6.3
6.9
This venue has not been rated yet.
7.6
7.3


1    None
2    None
5    None
0    None
9    None
7    None
6    None
4    None
3    None
8    None
Name: id, dtype: object

# The best choice for the venue
Scarborough Village

In [50]:
ScarboroughVillage_latitude = 43.744734; ScarboroughVillage_longitude = -79.239476
map_ScarboroughVillage = folium.Map(location = [ScarboroughVillage_latitude, ScarboroughVillage_longitude], zoom_start = 10)
folium.CircleMarker(
    [ScarboroughVillage_latitude, ScarboroughVillage_longitude],
    radius=5,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7).add_to(map_ScarboroughVillage)      
map_ScarboroughVillage